In [3]:
import pandas as pd
import numpy as np
import ast
import math
from pyvis.network import Network
import sys
sys.path.append('../')
from modules import utils, constants

#### The test df

In [4]:
model_name = 'dueling_dqn_per'
seed = 42
steps = int(10e7)

In [6]:
test_df = pd.read_csv(f'../test_dfs/{model_name}_risk_factor_{seed}_{steps}.csv')
test_df.head()

,index,episode_length,reward,y_pred,y_actual,trajectory,terminated,is_success
0,0.0,12.0,0.564819,1.0,1.0,"['ana', 'joint_involvement', 'anti_dsdna_antib...",0.0,1.0
1,1.0,13.0,0.501479,1.0,1.0,"['ana', 'joint_involvement', 'anti_dsdna_antib...",0.0,1.0
2,2.0,12.0,0.540303,1.0,1.0,"['ana', 'joint_involvement', 'anti_dsdna_antib...",0.0,1.0
3,3.0,10.0,0.646010,1.0,1.0,"['ana', 'joint_involvement', 'anti_dsdna_antib...",0.0,1.0
4,4.0,10.0,0.649264,1.0,1.0,"['ana', 'joint_involvement', 'anti_dsdna_antib...",0.0,1.0


In [7]:
utils.test(test_df.y_actual, test_df.y_pred)[0]

98.95714285714286

In [8]:
test_df[(test_df.y_pred==0) & (test_df.episode_length!=2)].episode_length.mean()

12.220062953332421

#### The other pertinent dfs

In [9]:
testing_df = pd.read_csv('../data/test_set_constant.csv')
testing_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,1,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
1,1,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1
2,1,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,1,0,0,0,1
3,1,1,0,0,0,0,0,0,1,0,...,0,0,0,0,1,0,0,1,0,1
4,1,0,0,0,0,0,0,0,1,0,...,1,0,0,0,0,1,0,0,0,1


In [10]:
training_df = pd.read_csv('../data/train_set_basic.csv')
training_df.head()

,ana,fever,leukopenia,thrombocytopenia,auto_immune_hemolysis,delirium,psychosis,seizure,non_scarring_alopecia,oral_ulcers,...,joint_involvement,proteinuria,anti_cardioliphin_antibodies,anti_β2gp1_antibodies,lupus_anti_coagulant,low_c3,low_c4,anti_dsdna_antibody,anti_smith_antibody,label
0,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
3,1,0,0,0,0,0,0,1,1,0,...,0,1,0,0,0,0,0,1,1,1
4,1,0,0,1,1,1,0,0,0,0,...,0,1,0,0,0,1,0,0,1,1


#### Possibly useful functions

In [11]:
def generate_tuple_dict(df):
    frequency_dict = {}
    for traj in df.trajectory:
        if traj in frequency_dict.keys():
            frequency_dict[traj] += 1
        else:
            frequency_dict[traj] = 1
    overall_tup_dict = {}
    for key, value in frequency_dict.items():
        new_key = ast.literal_eval(key)
        for tup in zip(new_key, new_key[1:]):
            if tup in overall_tup_dict.keys():
                overall_tup_dict[tup] += value
            else:
                overall_tup_dict[tup] = value
    return overall_tup_dict

In [12]:
def create_sankey_df(df):
    overall_tup_dict = generate_tuple_dict(df)
    sankey_df = pd.DataFrame()
    sankey_df['source'] = [i[0] for i in overall_tup_dict.keys()]
    sankey_df['target'] = [i[1] for i in overall_tup_dict.keys()]
    sankey_df['value'] = list(overall_tup_dict.values())
    sankey_df['link_type'] = sankey_df['target'].apply(lambda i: 'terminal' if i in ['No lupus', 'Lupus', 'Inconclusive diagnosis'] else 'non_terminal')
    return sankey_df

In [13]:
def draw_pyvis_network(pathways_df, pathway_type='network'): #second type is trajectory 
    pathways_df['type'] = 'directed'
    start_node = pathways_df.iloc[0]['source']
    all_nodes = list(set(pathways_df.source.unique().tolist() + pathways_df.target.unique().tolist()))
    terminal_nodes = list(set(pathways_df[pathways_df.link_type=='terminal'].target))
    non_terminal_nodes = [i for i in all_nodes if i not in terminal_nodes and i!=start_node]
    
    #change get_net to something else since it satnds for Game of Thrones
    got_net = Network(notebook=True, height='600px', width='100%', directed=True, cdn_resources='in_line')
    got_net.add_node(start_node, color='purple', size=20)
    got_net.add_nodes(non_terminal_nodes, size=[15]*len(non_terminal_nodes), color=['blue']*len(non_terminal_nodes))
    got_net.add_nodes(terminal_nodes, color=['green']*len(terminal_nodes), size=[20]*len(terminal_nodes))
    for src, target, value, edge_thresh in zip(pathways_df.source, pathways_df.target, pathways_df.value, pathways_df.edge_threshold):
        if pathway_type =='network':
            if value > threshold:
                if math.isnan(edge_thresh):
                    got_net.add_edge(src, target, value=value, color='red')
                else:
                    got_net.add_edge(src, target, value=value, color='red', label=str(int(edge_thresh)))
            else:
                got_net.add_edge(src, target, value=value, color='blue')
        elif pathway_type =='trajectory': #It's just one trajectory
            if isinstance(edge_thresh, str):
                got_net.add_edge(src, target, value=value, color='blue', label=edge_thresh)
            else:
                got_net.add_edge(src, target, value=value, color='blue', label=str(int(edge_thresh)))
        else:
            print('Unknown pathway type')
    return got_net

In [14]:
def get_threshold_value(df, source): #source e.g. 'ana'
    threshold_values = df[source].unique().tolist()
    if len(threshold_values) == 1:
        return int(threshold_values[0])
    else:
        return str(threshold_values)

In [15]:
def draw_one_trajectory(pred_df, trajectory): #pred_df = pred_no_lupus par exemple
    trajectory_df = pred_df[pred_df.trajectory == trajectory]
    trajectory_testing_df = testing_df.loc[trajectory_df.index]
    trajectory_pathways_df = create_sankey_df(trajectory_df)
    trajectory_pathways_df['edge_threshold'] = [get_threshold_value(trajectory_testing_df, source) for source in trajectory_pathways_df.source]
    trajectory_net = draw_pyvis_network(trajectory_pathways_df, 'trajectory')   
    return trajectory_net

##### shortest pathway

In [53]:
pred_lupus = test_df[test_df['y_pred']==1]
shortest_lupus_path = pred_lupus[pred_lupus.episode_length == pred_lupus.episode_length.min()].trajectory.unique()[0]
shortest_lupus_path

"['ana', 'joint_involvement', 'anti_dsdna_antibody', 'psychosis', 'Lupus']"

In [54]:
len(pred_lupus[pred_lupus.trajectory ==shortest_lupus_path])

82

In [55]:
shortest_lupus_path_net = draw_one_trajectory(pred_lupus, shortest_lupus_path)
shortest_lupus_path_net.show('Example.html')

Example.html


##### commonest pathway 

In [56]:
commonest_lupus_path = pred_lupus.trajectory.value_counts().index[1]
commonest_lupus_path

"['ana', 'joint_involvement', 'anti_dsdna_antibody', 'psychosis', 'low_c3', 'leukopenia', 'auto_immune_hemolysis', 'lupus_anti_coagulant', 'proteinuria', 'Lupus']"

In [57]:
len(pred_lupus[pred_lupus.trajectory ==commonest_lupus_path])

247

In [58]:
commonest_lupus_path_net = draw_one_trajectory(pred_lupus, commonest_lupus_path)
commonest_lupus_path_net.show('Example.html')

Example.html


##### longest pathways 

In [59]:
len(pred_lupus[pred_lupus.episode_length == pred_lupus.episode_length.max()].trajectory.value_counts())

28

In [60]:
longest_lupus_path1 = pred_lupus[pred_lupus.episode_length==pred_lupus.episode_length.max()].trajectory.value_counts().index[0]
longest_lupus_path2 = pred_lupus[pred_lupus.episode_length==pred_lupus.episode_length.max()].trajectory.value_counts().index[1]
# longest_lupus_path3 = pred_lupus[pred_lupus.episode_length==pred_lupus.episode_length.max()].trajectory.value_counts().index[2]
longest_lupus_path1

"['ana', 'joint_involvement', 'anti_dsdna_antibody', 'fever', 'psychosis', 'low_c3', 'proteinuria', 'non_scarring_alopecia', 'cutaneous_lupus', 'leukopenia', 'pleural_effusion', 'pericardial_effusion', 'lupus_anti_coagulant', 'delirium', 'thrombocytopenia', 'low_c4', 'seizure', 'oral_ulcers', 'anti_cardioliphin_antibodies', 'acute_pericarditis', 'anti_β2gp1_antibodies', 'auto_immune_hemolysis', 'anti_smith_antibody', 'Lupus']"

In [61]:
len(pred_lupus[pred_lupus.trajectory ==longest_lupus_path1])

8

In [62]:
longest_lupus_path1_net = draw_one_trajectory(pred_lupus, longest_lupus_path1)
longest_lupus_path1_net.show('Example.html')

Example.html


In [63]:
len(pred_lupus[pred_lupus.trajectory ==longest_lupus_path2])

7

In [64]:
longest_lupus_path2_net = draw_one_trajectory(pred_lupus, longest_lupus_path2)
longest_lupus_path2_net.show('Example.html')

Example.html


In [ ]:
#### Misdi

In [ ]:
longest_lupus_path3_net = draw_one_trajectory(pred_lupus, longest_lupus_path3)
longest_lupus_path3_net.show('Example.html')

In [ ]:
len(pred_lupus[pred_lupus.trajectory ==longest_lupus_path3])

In [65]:
pathway_samp = str(['ana', 'joint_involvement', 'anti_dsdna_antibody', 'low_c3', 'psychosis', 'anti_smith_antibody', 'pleural_effusion', 'proteinuria', 'leukopenia', 'fever', 'non_scarring_alopecia', 'low_c4', 'pericardial_effusion', 'cutaneous_lupus', 'seizure', 'delirium', 'lupus_anti_coagulant', 'thrombocytopenia', 'Lupus'])
pathway_samp

"['ana', 'joint_involvement', 'anti_dsdna_antibody', 'low_c3', 'psychosis', 'anti_smith_antibody', 'pleural_effusion', 'proteinuria', 'leukopenia', 'fever', 'non_scarring_alopecia', 'low_c4', 'pericardial_effusion', 'cutaneous_lupus', 'seizure', 'delirium', 'lupus_anti_coagulant', 'thrombocytopenia', 'Lupus']"

In [ ]:
a = pred_lupus[(pred_lupus.trajectory == pathway_samp)&(pred_lupus.y_actual!=pred_lupus.y_pred)]

In [69]:
draw_one_trajectory(pred_lupus, pathway_samp).show('Example.html')

Example.html
